# Import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn 

In [2]:
from yelpapi import YelpAPI
import os, json, math, time
from tqdm.notebook import tqdm_notebook

# Credentials and Accessing the API



In [3]:
# Load API Credentials
with open ('/Users/ERNESTO/.secret/yelp_api.json') as f:
    login = json.load(f)
    
# Instantiate YelpAPI variable

yelp_api = YelpAPI(login['api-key'], timeout_s = 5.0)

# Define Search

To allow us to easily perform different searches in the future, we will define variables for LOCATION and TERM set for our particular search conditions. Then, when we want to use a different location or term, we can just redefine these variables. This streamlines the code and makes it more readable and reproducible.

In [4]:
LOCATION = 'NY, NY'
TERM = 'Pizza'

# Create a results-in-progress JSON file, but only if it doesn't exist.

This is the file where your results will be saved. Note: you must rename your JSON_FILE for different queries to prevent confusing results from other searches. We recommend you include your search terms in the filename.

In [5]:
# Specifying JSON_FILE filename (can include a folder)
# include the search term in the filename
JSON_FILE = "Data/results_in_progress_NY_pizza.json"
JSON_FILE

'Data/results_in_progress_NY_pizza.json'

# Check if our JSON_FILE already exists. 

This will prevent us from accidentally overwriting an existing file.

If it doesn't exist:

Create any folders needed for the file path.

Save an empty list as JSON_File

In [6]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")


[i] Data/results_in_progress_NY_pizza.json not found. Saving empty list to file.


# Determine how many results are already in the file

Load the results file to determine the # of results we have previously retrieved. If you just created the file, you would expect it to be empty.

We will use this as our offset parameter for our API call. Even if this is your first API call, and the number is 0, we want to define "n_results" based on the length of "previous_results."

In [7]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')



- 0 previous results found.


# Figure out how many pages of results we will need

We will perform our first query to get our first page of results and the total number of results. We will then (via code) calculate how many pages we will need to retrieve all of our results.

In [8]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()



dict_keys(['businesses', 'total', 'region'])

In [9]:
len(results)

3

In [10]:
## How many results total?
total_results = results['total']
total_results



12400

In [11]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page



20

There are over 12400 businesses to retrieve from our API and we can get 20 results at a time (per "page").

We can calculate the # of results remaining by subtracting our offset (length of our previous results) from our total.

Then we can determining how many pages we will need by dividing the results by 20 (or whatever the value happens to be for results per page)

Note that we need to round up the number of pages in order to get all of the results. Even if there is only 1 result on the last page, we want to include that page! To do this we will use math.ceil.

In [12]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages



620

When this example was written, there were 12400 results and 20 results per page. 12400 /20 = 620. But what if we had 12410 results? Rounding UP would give us 561 pages. We expect the first 560 pages to have 20 results and the last page to have the final 10 results. Notice that we have assigned the number of pages as n_pages here. We will use this value in our next segment of code.

You can see that having to manually go through 620 pages would be quite time consuming and inefficient! First we are going to save the first page into our file, and then we will add on to it with a for loop.

# Add this page of results to .json file

Our API returns our results in JSON format, with the businesses in a list of dictionaries. We will append the first page of businesses to our previous_results (which is very likely empty) and then save to disk.

In [13]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)


# Set up a progress bar in our for loop.

To keep us informed about where we are in our loop, we will add a progress bar to our for loop.

TQDM is a package designed for adding animated progress bars to Python processes. It is not currently included in your dojo-env, so you are going to install it manually by opening a new Terminal/GitBash window and running the following command:

pip install tqdm

TQDM is easy to use and simply needs to know what we are looping through. If you wanted to test tqdm in action, but your loop is too fast, you can import time and use time.sleep to add a pause within your for loop. We will also use time.sleep when executing many API calls so that we do not overwhelm the server.

In [14]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 


  0%|          | 0/620 [00:00<?, ?it/s]

# For Loop to call each page
The loop below will iterate through each page of the results by starting at the appropriate offset. It will then append the results to the previous_results. This may take some time, so check out the progress bar!

In [15]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    # add a 200ms pause
    time.sleep(.2)



  0%|          | 0/620 [00:00<?, ?it/s]

YelpAPIError: VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.

# Oh, no! What happened?
If you examine the bottom of our error message,

YelpAPIError: VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.

It is telling us that we asked for too many results and that we can only get <= 1,000 results.

This is the limitation of using the free tier of Yelp's API. If we were to pay a monthly fee for better access, we would not hit this limitation. Unfortunately, there is no way to adjust our calls to skip those first 1,000. So we can only ever get the same first 1,000 results.

So what can we do about it now so that we run our code without error?

# Handling queries with >1000 Results
To get around this error, we can add an extra logic check to see if the length of movies we have so far (n_results) + the # of results in each page (results_per_page) is greater than 1,000.

If it is greater than 1,000 we will use break to end our loop early.

# Deleting Our Previous Results FIle
Let's give ourselves a fresh start with our new and improved loop. Let's delete our previous results file.
We could accomplish this manually or programmatically.
# The Manual Way
We could do this manually by using the Files page in Jupyter to find the file in our results folder and delete it. To delete a file with jupyter check the check box next to the file you want to delete.

You should now see additional buttons appear above the list of files.

Click on the red trash can icon to delete the file.

# The Programmatic Way
Just like we used the os module create new folders, we can also use it to delete (or remove) files.

We simply use the os.remove function and pass in our filename to delete.

We can then use our os.path.isfile function again to confirm the file no longer exists.

In [16]:
## delete file and confirm it no longer exits.
os.remove(JSON_FILE)
os.path.isfile(JSON_FILE)



False

Now that we've deleted the file, we need to re-run our code to create it.

This process is begging to be turned into a function so we can we easily repeat it.

While we are making it a function, why don't we go ahead and add the option to delete the JSON file it if already exists, just like we did above.

So let's make a create_json_file function that accepts the JSON_FILE filename as first argument and a second argument called delete_if_exists and set to it to False by default.

This way, it will not automatically delete previous search results. We will have to explicitly say delete_if_exists = True to do so

In [81]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[i] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else: print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  



IndentationError: unindent does not match any outer indentation level (<tokenize>, line 17)

Now that we have our new function, we can use it with delete_if_exists=True to delete our previous results and start over. We will also need to repeat the steps to recreate our n-results, total_results, results_per_page, and n_pages variables that we created before our first attempted loop.

In [82]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages



[i] Data/results_in_progress_NY_pizza.json not found. Saving empty list to file.
- 0 previous results found.


615

In [83]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)



  0%|          | 0/615 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


# After the loop has finished

Convert .json to dataframe
Load in the "results in progress" JSON file into a DataFrame:

In [84]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,zj8Lq1T8KIC5zwFief15jg,prince-street-pizza-new-york-2,Prince Street Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/I4gm7i...,False,https://www.yelp.com/biz/prince-street-pizza-n...,4662,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.72308755605564, 'longitude': -...","[pickup, delivery]",$,"{'address1': '27 Prince St', 'address2': None,...",+12129664100,(212) 966-4100,1961.877142
1,ysqgdbSrezXgVwER2kQWKA,julianas-brooklyn-3,Juliana's,https://s3-media2.fl.yelpcdn.com/bphoto/NVoLFl...,False,https://www.yelp.com/biz/julianas-brooklyn-3?a...,2584,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.70274718768062, 'longitude': -...",[delivery],$$,"{'address1': '19 Old Fulton St', 'address2': '...",+17185966700,(718) 596-6700,308.569844
2,v1DHGRNCH9247WLYoaoA9A,l-industrie-pizzeria-brooklyn,L'industrie Pizzeria,https://s3-media2.fl.yelpcdn.com/bphoto/SkMxUE...,False,https://www.yelp.com/biz/l-industrie-pizzeria-...,838,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.71162, 'longitude': -73.95783}",[delivery],$,"{'address1': '254 S 2nd St', 'address2': '', '...",+17185990002,(718) 599-0002,3145.016041
3,WG639VkTjmK5dzydd1BBJA,rubirosa-new-york-2,Rubirosa,https://s3-media4.fl.yelpcdn.com/bphoto/LuSzR8...,False,https://www.yelp.com/biz/rubirosa-new-york-2?a...,2846,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 40.722766, 'longitude': -73.996233}",[pickup],$$,"{'address1': '235 Mulberry St', 'address2': ''...",+12129650500,(212) 965-0500,1932.946770
4,WIhm0W9197f_rRtDziq5qQ,lombardis-pizza-new-york-4,Lombardi's Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/UZ6V_h...,False,https://www.yelp.com/biz/lombardis-pizza-new-y...,6491,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.7215934960083, 'longitude': -7...","[pickup, delivery]",$$,"{'address1': '32 Spring St', 'address2': '', '...",+12129417994,(212) 941-7994,1798.995978


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,qxOcH2ij2Erp1SKQfvuDdw,giovannis-pizzeria-and-restaurant-hoboken-6,Giovanni's Pizzeria & Restaurant,https://s3-media3.fl.yelpcdn.com/bphoto/l1oXpu...,False,https://www.yelp.com/biz/giovannis-pizzeria-an...,150,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.5,"{'latitude': 40.7436186428648, 'longitude': -7...","[pickup, delivery]",$,"{'address1': '603 Washington St', 'address2': ...",+12017144232,(201) 714-4232,5145.239295
996,db2QLlK-0xL7Vq0jG-CcJQ,café-bello-bayonne-2,Café Bello,https://s3-media3.fl.yelpcdn.com/bphoto/9aQiDN...,False,https://www.yelp.com/biz/caf%C3%A9-bello-bayon...,188,"[{'alias': 'italian', 'title': 'Italian'}]",4.5,"{'latitude': 40.68387, 'longitude': -74.10426}",[delivery],$$,"{'address1': '1044 Avenue C', 'address2': '', ...",+12014377538,(201) 437-7538,9580.774662
997,gED3ep4dX5FP_FYVTkokIQ,salvatores-of-soho-bronx,Salvatores of Soho,https://s3-media3.fl.yelpcdn.com/bphoto/aqrp4F...,False,https://www.yelp.com/biz/salvatores-of-soho-br...,331,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 40.8875232, 'longitude': -73.90704}",[delivery],$$,"{'address1': '3738 Riverdale Ave', 'address2':...",+17185487499,(718) 548-7499,21553.151257
998,MbNGMOpdAks5c9dNeQpXkA,de-palma-pizzeria-and-ristorante-union-city,De Palma Pizzeria & Ristorante,https://s3-media2.fl.yelpcdn.com/bphoto/yZ1JIk...,False,https://www.yelp.com/biz/de-palma-pizzeria-and...,80,"[{'alias': 'pizza', 'title': 'Pizza'}]",3.5,"{'latitude': 40.763534, 'longitude': -74.033157}",[delivery],$$,"{'address1': '1814 New York Ave', 'address2': ...",+12018648877,(201) 864-8877,7246.922143
999,mRCpatZsFjQhbMxZQZcS-A,porto-bello-astoria-2,Porto Bello,https://s3-media4.fl.yelpcdn.com/bphoto/WSpU69...,False,https://www.yelp.com/biz/porto-bello-astoria-2...,223,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.5,"{'latitude': 40.770352, 'longitude': -73.904303}","[pickup, delivery]",$$,"{'address1': '43-18 Ditmars Blvd', 'address2':...",+17182048766,(718) 204-8766,10467.821719


# Check for duplicates
Check for and remove any duplicate results.

In [85]:
# check for duplicate results
final_df.duplicated().sum()



TypeError: unhashable type: 'list'

In [87]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()



81

In [88]:
## Drop duplicate ids and confirm there are no more duplicates
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()


0

# Save the final DataFrame to a .csv (or a .csv.gz if its too big for the GitHub file size limit).

In [89]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_NY_pizza.csv.gz', compression='gzip',index=False)

